### Lagrange interpolation approach

Consider gate $U(x) = e^{-ixG}$

$G$ has $n$ distinguish eigenvalues $\lambda_k, k=\{0,1,...,n-1\}$

$e^{-i x \boldsymbol{G}}=\sum_{k=0}^{n-1} e^{-i x \lambda_{k}} \prod_{l=0, l \neq k}^{n-1} \frac{\boldsymbol{G}-\lambda_{l} \boldsymbol{I}}{\lambda_{k}-\lambda_{l}} = f(\boldsymbol{G})=\Lambda_0 I + \Lambda_1 G + ... + \Lambda_{n-1} G^{n - 1}$

#### 1. Calculate coeff ${\Lambda_i}$ of a gate

Here we treat $\boldsymbol{G}$ as a variable. The input is only x and eigenvalues ${\lambda_i}$

In [1]:
import numpy as np
import base
# Input
x = -np.pi / 2
lambdas = [-1, 0, 1]
# Polynomial presentation
fG = base.calculate_Lambda(lambdas, x)
# Output
print("Lambda_i: ", fG.coeff)

Lambda_i:  [(1+0j), 1j, (-1+0j)]


#### 2. Using more term to correct parameter-shift rule

In [2]:
alpha = np.pi / 4
# beta = np.pi / 2
# gamma = np.pi / 3
lambdas = [-1, 0, 1]
# lambdas = [-3/2, -1/2, 1/2, 3/2]
# lambdas = [-2, -1, 0, 1, 2]
# lambdas = [-3, -2, -1, 1, 2, 3]
# lambdas = [-3, -2, -1, 0, 1, 2, 5]
delta_Malpha = base.calculate_Lambda_matrix(lambdas, alpha) - base.calculate_Lambda_matrix(lambdas, -alpha)
#delta_Mbeta = base.calculate_Lambda_matrix(lambdas, beta) - base.calculate_Lambda_matrix(lambdas, -beta)
#delta_Mgamma = base.calculate_Lambda_matrix(lambdas, gamma) - base.calculate_Lambda_matrix(lambdas, -gamma)
d1 = 1/2
d2 = (-np.sqrt(2) + 1) / 4
# T_alpha = upper_matrix(delta_Malpha)
# T_beta = upper_matrix(delta_Mbeta)
# T_gamma = upper_matrix(delta_Mgamma)


In [3]:
print(np.round(delta_Malpha, 3))
print(base.upper_matrix(delta_Malpha))

[[ 0.+0.j     0.-1.414j  0.+0.j   ]
 [ 0.+1.414j  0.+0.j    -0.-0.414j]
 [ 0.+0.j    -0.+0.414j  0.+0.j   ]]
[[-0.-0.41421356j]]


In [12]:
Ts = []
deltas = []
thetas = [np.pi/2, np.pi]
dim_d = 2 # int(len(lambdas)**2/4) - 1 
for i in range(0, dim_d):
    theta = np.random.uniform(0, 2*np.pi)
    delta = (base.calculate_Lambda_matrix(lambdas, theta) - base.calculate_Lambda_matrix(lambdas, -theta))
    thetas.append(theta)
    deltas.append(delta)
    Ts.append(base.upper_matrix(delta))

In [93]:
Ts = []
deltas = []
thetas = [np.pi/4, 3*np.pi/4]
dim_d = 2 # int(len(lambdas)**2/4) - 1 
for i in range(0, dim_d):
    delta = (base.calculate_Lambda_matrix(lambdas, thetas[i]) - base.calculate_Lambda_matrix(lambdas, -thetas[i]))
    deltas.append(delta)
    Ts.append(base.upper_matrix(delta))

In [94]:
deltas[0]

array([[ 0.+0.j        ,  0.-1.41421356j,  0.+0.j        ],
       [ 0.+1.41421356j,  0.+0.j        , -0.-0.41421356j],
       [ 0.+0.j        , -0.+0.41421356j,  0.+0.j        ]])

In [95]:
deltas[1]

array([[ 0.+0.j        ,  0.-1.41421356j,  0.+0.j        ],
       [ 0.+1.41421356j,  0.+0.j        , -0.-2.41421356j],
       [ 0.+0.j        , -0.+2.41421356j,  0.+0.j        ]])

In [96]:
T = Ts[0]
for i in range(1, len(Ts)):
    T = np.hstack((T, Ts[i]))

In [97]:
from scipy.linalg import null_space

init_rcond = 1
while True:
    d = (null_space(T, rcond = init_rcond))
    if d.shape[1] != 1:
        init_rcond /= 10
    else:
        break

In [98]:
d

array([[-0.3240665 -0.93079827j],
       [ 0.05560102+0.15969974j]])

In [99]:
thetas

[0.7853981633974483, 2.356194490192345]

In [100]:
T @ d

array([[0.+4.71844785e-16j]])

In [101]:
sumMatrix = d[0] * deltas[0]
for i in range(1, len(Ts)):
    sumMatrix += d[i] * deltas[i]
print(np.round(sumMatrix, 3))

[[ 0.  +0.j   -1.09+0.38j  0.  +0.j  ]
 [ 1.09-0.38j  0.  +0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  -0.j    0.  +0.j  ]]


Theo công thức phải chia thêm hệ số (2j)

In [113]:
d_t = d / ((2j)*(-1.09+0.38j))

In [114]:
d_t

array([[ 0.42691009+1.76683978e-04j],
       [-0.07324619-3.03141781e-05j]])

d_t giống với kết quả của [Pennylane](https://docs.pennylane.ai/en/stable/code/api/pennylane.CRY.html)

In [115]:
(1 - np.sqrt(2)) / (4*np.sqrt(2))

-0.07322330470336313

In [91]:
sumMatrix = d_t[0] * deltas[0]
for i in range(1, len(Ts)):
    sumMatrix += d_t[i] * deltas[i]
print(np.round(sumMatrix, 3))

[[ 0.+0.j    -1.+0.001j  0.+0.j   ]
 [ 1.-0.001j  0.+0.j     0.-0.j   ]
 [ 0.+0.j    -0.+0.j     0.+0.j   ]]


In [12]:
alpha = np.pi / 4
beta = 3*np.pi / 4
lambdas = [-1, 0, 1]

delta_Malpha = base.calculate_Lambda_matrix(lambdas, alpha) - base.calculate_Lambda_matrix(lambdas, -alpha)
delta_Mbeta = base.calculate_Lambda_matrix(lambdas, beta) - base.calculate_Lambda_matrix(lambdas, -beta)

d1 = (np.sqrt(2) + 1)/(4*np.sqrt(2))
d2 = (-np.sqrt(2) + 1)/(4*np.sqrt(2))

print(d1*delta_Malpha + d2*delta_Mbeta)

[[0.+0.00000000e+00j 0.-5.00000000e-01j 0.+0.00000000e+00j]
 [0.+5.00000000e-01j 0.+0.00000000e+00j 0.+5.55111512e-17j]
 [0.+0.00000000e+00j 0.-5.55111512e-17j 0.+0.00000000e+00j]]
